<a href="https://colab.research.google.com/github/sunghyunjun/kaggle-lecr/blob/main/lecr_sbert_simcse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check colab gcp, gpu environment

In [ ]:
# Check colab's gcp region
!curl ipinfo.io

{
  "ip": "34.69.182.5",
  "hostname": "5.182.69.34.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS396982 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
gcs_bucket = "sunghyun-kaggle-lecr-us"
# gcs_bucket = "sunghyun-kaggle-lecr-eu"
# gcs_bucket = "sunghyun-kaggle-lecr-asia"

In [ ]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-d6496829-0e68-f681-11dd-d7381a146a9a)


In [ ]:
import multiprocessing
print(multiprocessing.cpu_count()) # Count the number of cores in a computer

12


# GCP personal setting for gcs, kaggle, neptune.ai

In [ ]:
# Authenticate to GCP
from google.colab import auth
auth.authenticate_user()

In [ ]:
# GCP secret manager, after run pip, notebook should be restarted
# !pip3 install --user google-cloud-secret-manager==2.5.0
!pip3 install --user google-cloud-secret-manager
from google.cloud import secretmanager


def get_secret(project_id, secret_name):
    """
    secret_name => To be replaced with your secret name
    project_id  => To be replaced with your GCP Project
    """

    # Create a Client:
    client = secretmanager.SecretManagerServiceClient()
    # secret_name = "kaggle" # => To be replaced with your secret name
    # project_id = 'otto-recsys' # => To be replaced with your GCP Project

    # Forge the path to the latest version of your secret with an F-string:
    resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest" 

    # Get your secret :
    response = client.access_secret_version(request={"name": resource_name})
    secret_string = response.payload.data.decode('UTF-8')

    # Tada ! you secret is in the secret_string variable!
    return secret_string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Install gcsfuse

In [ ]:
!pip install gcsfs --upgrade -q
!pip install psutil --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 6.6 MB/s eta 0:00:00


In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  48400      0 --:--:-- --:--:-- --:--:-- 48400
OK
26 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 26 not upgraded.
Need to get 13.8 MB of archives.
After this operation, 32.2 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 128275 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.42.1_amd64.deb ...
Unpacking gcsfuse (0.42.1) ...
Setting up gcsfuse (0.42.1) ...


In [ ]:
!mkdir ./mount_checkpoint
!gcsfuse {gcs_bucket} ./mount_checkpoint

2023/03/13 14:25:14.266332 Start gcsfuse/0.42.1 (Go version go1.19.5) for app "" using mount point: /content/mount_checkpoint
2023/03/13 14:25:14.279114 Opening GCS connection...
2023/03/13 14:25:14.450488 Mounting file system "sunghyun-kaggle-lecr-us"...
2023/03/13 14:25:14.451060 File system has been successfully mounted.


# Kaggle API credentials

In [ ]:
with open('kaggle.json', 'a') as fp:
    fp.write(get_secret("kaggle-lecr", "kaggle"))

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


# Secret for Neptune logging

In [ ]:
import os
os.environ["NEPTUNE_PROJECT"] = "sunghyun.jun/lecr"
os.environ["NEPTUNE_API_TOKEN"] = get_secret("kaggle-lecr", "neptune")
os.environ["NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE"] = "TRUE"

In [ ]:
%env NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE='TRUE'

env: NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE='TRUE'


# Download Kaggle dataset

In [ ]:
!kaggle competitions download -c learning-equality-curriculum-recommendations -p dataset

 99% 252M/254M [00:01<00:00, 131MB/s]
100% 254M/254M [00:01<00:00, 137MB/s]


In [ ]:
!unzip dataset/learning-equality-curriculum-recommendations.zip -d dataset

Archive:  dataset/learning-equality-curriculum-recommendations.zip
  inflating: dataset/content.csv     
  inflating: dataset/correlations.csv  
  inflating: dataset/sample_submission.csv  
  inflating: dataset/topics.csv      


In [ ]:
# !kaggle datasets download shkanda/sbert-models -p sbert-models
!kaggle datasets download inversion/sentence-transformers-222 -p sentence-transformers-222

 96% 490M/509M [00:04<00:00, 145MB/s]
100% 509M/509M [00:04<00:00, 128MB/s]


In [ ]:
# !unzip sbert-models/sbert-models.zip -d sbert-models
!unzip sentence-transformers-222/sentence-transformers-222.zip -d sentence-transformers-222

Archive:  sentence-transformers-222/sentence-transformers-222.zip
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/HEAD  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/config  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/description  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/applypatch-msg.sample  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/commit-msg.sample  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/fsmonitor-watchman.sample  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/post-checkout  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/post-commit  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/post-merge  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/post-update.sample  
  inflating: sentence-transformers-222/all-MiniLM-L6-v2/.git/hooks/pre-applypatch.sample  
  inflating: sentence-tra

In [ ]:
!kaggle kernels output sunghyunjun/lecr-prepare-dataset-all -p dataset

Output file downloaded to dataset/encoder.pkl
Output file downloaded to dataset/fold_0/train_fold_0.parquet
Output file downloaded to dataset/fold_0/valid_fold_0.parquet
Output file downloaded to dataset/fold_1/train_fold_1.parquet
Output file downloaded to dataset/fold_1/valid_fold_1.parquet
Output file downloaded to dataset/fold_2/train_fold_2.parquet
Output file downloaded to dataset/fold_2/valid_fold_2.parquet
Output file downloaded to dataset/fold_3/train_fold_3.parquet
Output file downloaded to dataset/fold_3/valid_fold_3.parquet
Output file downloaded to dataset/fold_4/train_fold_4.parquet
Output file downloaded to dataset/fold_4/valid_fold_4.parquet
Kernel log downloaded to dataset/lecr-prepare-dataset-all.log 


In [ ]:
# !kaggle kernels output sunghyunjun/my-kaggle-notebook -p dataset


# Install packages

In [ ]:
!pip install -U sentence-transformers
!pip install -U polars
!pip install neptune
!pip install scikit-learn==1.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=da985484b0cc82c67141daf709287b3bcb7ee89e7d7e094e7a5efb849fef3e7d
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import packages

In [ ]:
import logging
import math
import pickle
from random import sample

import neptune
from neptune.integrations.python_logger import NeptuneHandler

import numpy as np

import polars as pl
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from sentence_transformers.evaluation import SentenceEvaluator
from sentence_transformers import LoggingHandler

import torch
from torch.utils.data import DataLoader

from tqdm.autonotebook import trange

In [ ]:
class LECROrdinalEncoders:
    def __init__(self):
        self.encoder = {}
        self.columns = {}
        
        self.columns["topic"] = [
            "channel_topic",
            "category_topic",
            "language_topic",
        ]
        self.columns["content"] = [
            "kind_content",
            "language_content",
            "license_content",
        ]
        self.columns["both"] = self.columns["topic"] + self.columns["content"]
        
        for col in self.columns["both"]:

            # language_topic and language_content have to use same ordinal encoder
            if col == "language_content":
                continue
            self.encoder[col] = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
        
    def fit(self, df, mode="topic"):
        if mode not in self.columns.keys():
            raise Exception(f"mode should be one of {self.columns.keys()}")

        for col in self.columns[mode]:
            if col == "language_content":
                print("OrdinalEncoder of language_topic column will be used for language_content column.")
                continue
                
            self.encoder[col].fit(df[col].to_numpy().reshape(-1, 1))

    def transform(self, df, mode="topic"):
        if mode not in self.columns.keys():
            raise Exception(f"mode should be one of {self.columns.keys()}")

        for col in self.columns[mode]:
            if col == "language_content":
                print("OrdinalEncoder of language_topic column will be used for language_content column.")
                s = self.encoder["language_topic"].transform(df[col].to_numpy().reshape(-1, 1))
            else: 
                s = self.encoder[col].transform(df[col].to_numpy().reshape(-1, 1))
                
            s = np.squeeze(s)
            s = pl.Series(s)
            df.replace(col, s)
    
    def fit_transform(self, df, mode="topic"):
        if mode not in self.columns.keys():
            raise Exception(f"mode should be one of {self.columns.keys()}")

        for col in self.columns[mode]:
            if col == "language_content":
                print("OrdinalEncoder of language_topic column will be used for language_content column.")
                s = self.encoder["language_topic"].transform(df[col].to_numpy().reshape(-1, 1))
            else:
                s = self.encoder[col].fit_transform(df[col].to_numpy().reshape(-1, 1))
                
            s = np.squeeze(s)
            s = pl.Series(s)
            df.replace(col, s)
            
    def save(self, filename="encoder.pkl"):
        with open(filename, "wb") as f:
            pickle.dump(self, f)

In [ ]:
def add_suffix(df, mode="topic"):
    columns = ["topic", "content"]
    if mode not in columns:
        raise Exception(f"mode should be one of {columns}")
        
    columns_suffix = {col: col + "_" + mode for col in df.columns}
    df = df.rename(columns_suffix)
    
    return df

# Logging

In [ ]:
run = neptune.init_run()

npt_handler = NeptuneHandler(run=run)

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler(), npt_handler],
                    force=True)

<ipython-input-13-d0623ad26aa5>:1: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run()


https://app.neptune.ai/sunghyun.jun/lecr/e/LEC-57


# Preparing

In [ ]:
MAX_LEN = 10000

tokens = ["[LECR_SEP]"]

In [ ]:
topics_path = "./dataset/topics.csv"
content_path = "./dataset/content.csv"

model_path = "./sentence-transformers-222/all-MiniLM-L6-v2"
encoders_path = "./dataset/encoder.pkl"

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
topics_df = pl.read_csv(topics_path)
content_df = pl.read_csv(content_path)

In [ ]:
topics_df = add_suffix(topics_df, mode="topic")
content_df = add_suffix(content_df, mode="content")

In [ ]:
with open(encoders_path, "rb") as f:
    encoders = pickle.load(f)

In [ ]:
encoders.transform(topics_df, mode="topic")
encoders.transform(content_df, mode="content")

OrdinalEncoder of language_topic column will be used for language_content column.


In [ ]:
topics_df = topics_df.with_columns([
    pl.concat_str([
        pl.col("language_topic").cast(int),
        pl.col("title_topic").fill_null(""),
        pl.col("description_topic").fill_null("").str.slice(0, MAX_LEN),
    ],
    separator="[LECR_SEP]",
    ).alias("sentence_topic")
])

In [ ]:
content_df = content_df.with_columns([
    pl.concat_str([
        pl.col("language_content").cast(int),
        pl.col("title_content").fill_null(""),
        pl.col("description_content").fill_null(""),
        pl.col("text_content").fill_null("").str.slice(0, MAX_LEN),
    ],
    separator="[LECR_SEP]",
    ).alias("sentence_content")
])

In [ ]:
topics_df.head()

id_topic,title_topic,description_topic,channel_topic,category_topic,level_topic,language_topic,parent_topic,has_content_topic,sentence_topic
str,str,str,f64,f64,i64,f64,str,bool,str
"""t_00004da3a1b2...","""Откриването на...","""Изследване на ...",0.0,1.0,4,2.0,"""t_16e29365b50d...",true,"""2[LECR_SEP]Отк..."
"""t_000095e03056...","""Unit 3.3 Enlar...",null,118.0,0.0,2,4.0,"""t_aa32fb6252dc...",false,"""4[LECR_SEP]Uni..."
"""t_00068291e9a4...","""Entradas e saí...","""Entenda um pou...",98.0,1.0,4,19.0,"""t_d14b6c2a2b70...",true,"""19[LECR_SEP]En..."
"""t_00069b63a70a...","""Transcripts""",null,78.0,1.0,3,4.0,"""t_4054df11a74e...",true,"""4[LECR_SEP]Tra..."
"""t_0006d41a73a8...","""Графики на екс...","""Научи повече з...",0.0,1.0,4,2.0,"""t_e2452e21d252...",true,"""2[LECR_SEP]Гра..."


In [ ]:
content_df.head()

id_content,title_content,description_content,kind_content,text_content,language_content,copyright_holder_content,license_content,sentence_content
str,str,str,f64,str,f64,str,f64,str
"""c_00002381196d...","""Sumar números ...","""Suma 48,029+23...",4.0,null,5.0,null,7.0,"""5[LECR_SEP]Sum..."
"""c_000087304a9e...","""Trovare i fatt...","""Sal trova i fa...",4.0,null,10.0,null,7.0,"""10[LECR_SEP]Tr..."
"""c_0000ad142ddb...","""Sumar curvas d...","""Cómo añadir cu...",4.0,null,5.0,null,7.0,"""5[LECR_SEP]Sum..."
"""c_0000c03adc8d...","""Nado de aproxi...","""Neste vídeo vo...",1.0,""" Nado de apro...",19.0,"""Sikana Educati...",2.0,"""19[LECR_SEP]Na..."
"""c_00016694ea2a...","""geometry-m3-to...","""geometry-m3-to...",1.0,"""Estándares Com...",5.0,"""Engage NY""",3.0,"""5[LECR_SEP]geo..."


In [ ]:
len(topics_df), len(content_df)

(76972, 154047)

In [ ]:
model = SentenceTransformer(model_path, device=device)
word_embedding_model = model._first_module()
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=True)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

2023-03-13 15:23:43 - Load pretrained SentenceTransformer: ./sentence-transformers-222/all-MiniLM-L6-v2


Embedding(30523, 384)

In [ ]:
model.get_max_seq_length()

256

In [ ]:
class LossEvaluator(SentenceEvaluator):

    def __init__(self, neptune_run, loader, loss_model: torch.nn.Module = None, name: str = '',
                 show_progress_bar: bool = False):

        """
        Evaluate a model based on the loss function.
        The returned score is loss value.
        :param loader: Data loader object
        :param loss_model: loss module object
        :param name: Name for the output
        :param show_progress_bar: If true, prints a progress bar
        """

        logger = logging.getLogger(__name__)

        self.run = neptune_run
        self.loader = loader
        self.name = name
        self.loss_model = loss_model

        # move model to gpu:  lidija-jovanovska
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        loss_model.to(self.device)

        if show_progress_bar is None:
            show_progress_bar = (
                    logger.getEffectiveLevel() == logging.INFO or logger.getEffectiveLevel() == logging.DEBUG)
        self.show_progress_bar = show_progress_bar

    def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1) -> float:

        self.loss_model.eval()

        loss_value = 0
        self.loader.collate_fn = model.smart_batching_collate
        num_batches = len(self.loader)
        data_iterator = iter(self.loader)

        with torch.no_grad():
            for _ in trange(num_batches, desc="Iteration", smoothing=0.05, disable=not self.show_progress_bar):
                sentence_features, labels = next(data_iterator)
                # move data to gpu
                for i in range(0, len(sentence_features)):
                    for key, value in sentence_features[i].items():
                        sentence_features[i][key] = sentence_features[i][key].to(self.device)
                labels = labels.to(self.device)
                loss_value += self.loss_model(sentence_features, labels).item()

        final_loss = loss_value / num_batches

        self.run["val_loss"].append(final_loss)

        self.loss_model.zero_grad()
        self.loss_model.train()

        return final_loss

In [ ]:
# Define a list with sentences (1k - 100k sentences)
sentences = topics_df["sentence_topic"].to_list() + content_df["sentence_content"].to_list()

In [ ]:
sentences[0]

'2[LECR_SEP]Откриването на резисторите[LECR_SEP]Изследване на материали, които предизвикват намаление в отклонението, когато се свържат последователно с нашия измервателен уред. '

In [ ]:
word_embedding_model.tokenizer(sentences[0])

{'input_ids': [101, 1016, 30522, 1193, 22919, 23925, 16856, 10325, 25529, 28995, 15290, 22919, 14150, 1192, 10260, 1195, 15290, 29744, 10325, 29747, 22919, 14150, 16856, 10325, 22919, 15290, 30522, 1188, 29744, 29747, 29436, 15290, 29742, 25529, 28995, 15290, 1192, 10260, 1191, 10260, 22919, 15290, 16856, 10325, 10260, 29436, 10325, 1010, 1189, 14150, 10325, 22919, 14150, 1194, 16856, 15290, 29742, 10325, 29744, 25529, 10325, 23925, 25529, 10260, 22919, 1192, 10260, 29745, 10260, 29436, 15290, 18947, 10325, 15290, 1182, 1193, 22919, 23925, 29436, 14150, 18947, 15290, 18947, 10325, 15290, 22919, 14150, 1010, 1189, 14150, 29741, 10260, 22919, 14150, 1196, 15290, 1196, 25529, 29755, 16856, 29743, 10260, 22919, 1194, 14150, 29747, 29436, 15290, 29742, 19259, 10260, 22919, 15290, 29436, 18947, 14150, 1196, 1192, 10260, 29753, 23483, 1188, 29744, 29745, 15290, 16856, 25529, 10260, 22919, 15290, 29436, 15290, 18947, 1198, 16856, 15290, 29742, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0

In [ ]:
train_sentences, valid_sentences = train_test_split(sentences, test_size=0.1)

In [ ]:
# Convert train sentences to sentence pairs
train_data = [InputExample(texts=[s, s]) for s in train_sentences]

# DataLoader to batch your data
train_dataloader = DataLoader(train_data, batch_size=192, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
valid_data = [InputExample(texts=[s, s]) for s in valid_sentences]
valid_dataloader = DataLoader(valid_data, batch_size=256, shuffle=False)
valid_evaluator = LossEvaluator(run, valid_dataloader, loss_model=train_loss, name='dev', show_progress_bar=True)

In [ ]:
print(len(train_sentences), len(valid_sentences))

207917 23102


# Training

In [ ]:
num_epochs = 3

In [ ]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2023-03-13 15:23:48 - Warmup-steps: 325


In [ ]:
# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=valid_evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    optimizer_params={"lr": 2e-05}, # 2e-5
    evaluation_steps=500,
    save_best_model=True,
    show_progress_bar=True,
    checkpoint_path="./mount_checkpoint/simcse-1",
    checkpoint_save_steps=500,
    # checkpoint_save_total_limit=10
)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1083 [00:00<?, ?it/s]

Iteration:   0%|          | 0/91 [00:00<?, ?it/s]

2023-03-13 15:31:05 - Save model to ./mount_checkpoint/simcse-1/500


Iteration:   0%|          | 0/91 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/91 [00:00<?, ?it/s]

2023-03-13 16:08:34 - Save model to ./mount_checkpoint/simcse-1/3000


Iteration:   0%|          | 0/91 [00:00<?, ?it/s]

Iteration:   0%|          | 0/91 [00:00<?, ?it/s]

2023-03-13 16:13:16 - Save model to ./mount_checkpoint/simcse-1/3249


In [ ]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/sunghyun.jun/lecr/e/LEC-57/metadata
